In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

import dotenv
import torch
import matplotlib.pyplot as plt
import mlflow
import os
from torch.utils.data import DataLoader
from intra import IntrA
from models.pointnet import PointNetCls
from models.pointnetcls import PointNet
from models.pointnet2 import PointNet2
from models.pointconv import PointConvDensityClsSsg
from utils import train_model, train_kfold_intra, eval_model_classification


os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

dotenv.load_dotenv()  # load the MLflow http authentication parameters
mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))

dev = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(dev)

mps


In [2]:
if False:
    # train pointnet for a single epoch to test utils on a slow laptop
    dataset = IntrA(
        "./data",
        npoints=1024,
        exclude_seg=True,
        norm=True,
        dataset="generated",
    )

    trn, tst = torch.utils.data.random_split(dataset, [0.8, 0.2], torch.Generator().manual_seed(42))
 
    model = PointNet(k=2)

    train_dl = DataLoader(trn, batch_size=8, drop_last=True)
    test_dl = DataLoader(tst, batch_size=8, drop_last=True)

    train_model(model, train_dl, test_dl, epochs=1, checkpoint_epoch=50, model_name="PointNet", norm=False)

In [3]:
train_kfold_intra(
    PointNet,
    {"k":2},
    epochs=1,
    model_name="PointNet",
    batch_size=8,
    num_workers=2,
    intra_root="./data",
    npoints=1024,
    norm=True,
    pointconv=False,
    splits="./file_splits/cls/",
    exclude_seg=True
)


F1:


/Users/jventers/miniconda3/envs/masters-env/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


TypeError: forward() takes 2 positional arguments but 3 were given